In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np

df = pd.read_csv('filtered_nn_data.csv')

df = df[['date', 'uv', 'buyer', 'gmv', 'orders']].copy()

print(df)
# df['date'] = pd.to_datetime(df['date'])

# Assuming your DataFrame is named df
# Sort data by date
df = df.sort_values(by='date')

# Create lag features (e.g., lag 1 day)
df['lag_1_day'] = df['order'].shift(1)

# Initialize and train the XGBoost model using all available data
X = df[['lag_1_day']].dropna()
y = df['order'].dropna()
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
model.fit(X, y)

# Create a DataFrame for forecasting
forecast_df = pd.DataFrame(columns=['date', 'order'])

# Set the initial date to the last date in your dataset
initial_date = df['date'].max()

# Forecast for 365 days into the future
for day in range(1, 366):
    # Create a DataFrame with the lagged order value for prediction
    prediction_data = pd.DataFrame({'lag_1_day': [y.iloc[-1]]})
    
    # Predict the order for the next day
    next_day_order = model.predict(prediction_data)
    
    # Update the forecast DataFrame with the date and predicted order
    forecast_df = forecast_df.append({'date': initial_date + pd.DateOffset(day), 'order': next_day_order[0]}, ignore_index=True)
    
    # Update the lagged order value for the next iteration
    y = y.append(pd.Series(next_day_order[0]), ignore_index=True)

# Display the forecasted orders for 365 days
print(forecast_df)
